In [ ]:
consumer_key = "rnMsljHPMOTTh5ajgQ6UglBeo"
consumer_secret = "wkEUaXT5OKj8C9dyE5NHqU1NQQhU501IrOvxbpJhBJueMUO1hH"
access_token = "270895064-sGfSXEJe5I4vsF1jUQcT35cfeFASwrHXVCrsv6J8"
access_token_secret = "Y04psn3gkalQBUWrvuGrlWphIoQfYD2UUV9mjihaubZOa"